In [52]:
import cv2
import numpy as np
import os
from scipy import ndimage
from collections import Counter

def process_image(img_path):
    try:
        # Resmi cv2 ile okur
        img = cv2.imread(img_path)
        if img is None:
            raise Exception(f"!Hata: Resim okunamadı - {img_path}!")

        # Resmi 128x128 olarak boyutlandırır ve grayscale'e çevirir
        gray_img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (128, 128))

        # Resmin türevini hesaplar
        img_derivative = np.abs(ndimage.sobel(gray_img, axis=0, mode='constant')) + np.abs(ndimage.sobel(gray_img, axis=1, mode='constant'))

        # Flatten, normalize et ve float16'ya çevir
        img_vector = (img_derivative - np.min(img_derivative)) / (np.max(img_derivative) - np.min(img_derivative))
        img_vector = img_vector.flatten().astype(np.float16)
    except Exception as processing_error:
        print(f"!Resmi işlerken hata oluştu: {processing_error}!")
        return None

    return img_vector

def load_data(data_folders):
    x_data = []
    y_data = []

    for label, folder_path in enumerate(data_folders, start=1):
        try:
            if os.path.exists(folder_path):
                for filename in os.listdir(folder_path):
                    image_path = os.path.join(folder_path, filename)

                    vector = process_image(image_path)
                    if vector is not None:
                        x_data.append(vector)
                        y_data.append(label)
        except Exception as loading_error:
            print(f"! {folder_path} klasöründen veri alınırken hata oluştu: {loading_error} !")

    return np.array(x_data), np.array(y_data)

# Train datası içeren üç klasörün pathleri
folders = [
    "/content/train/cellphone",
    "/content/train/flamingo",
    "/content/train/Motorbikes"
]

try:
    x_data, y_data = load_data(folders)

    # KNN Fonksiyonu
    def calculate_distance(a, b):
        return np.linalg.norm(a - b)

    def k_nearest_neighbors(x_data, y_data, test_sample, k=5):
        distances = [np.linalg.norm(test_sample - x) for x in x_data]  # Öklidyen mesafeleri hesaplar
        nearest_indices = np.argsort(distances)[:k]

        nearest_labels = [y_data[indicate] for indicate in nearest_indices]

        label_counts = Counter(nearest_labels)
        most_common_label = label_counts.most_common(1)[0][0]

      #  print(f"Classification results (numeric): {label_counts}")  #çıktıları kontrol etmek için
      #  print(f"Distances: {distances}")  #distanceları kontrol etmek için

        return most_common_label

    # Test part
    test_image_path = "/content/test/Motorbikes/image_0696.jpg"  # Test resminin pathi, test edilmek istenen resim yolu kopyalanmalı
    test_sample_vector = process_image(test_image_path)
    test_sample_vector = (test_sample_vector - np.min(test_sample_vector)) / (np.max(test_sample_vector) - np.min(test_sample_vector))
    test_sample_vector = test_sample_vector.flatten().astype(np.float16)

    try:
        predicted_label = k_nearest_neighbors(x_data, y_data, test_sample_vector)

        # Predict edilen etiketi yazdırır
        if predicted_label == 1:
            print("Tahmin edilen kategori: cellphone")
        elif predicted_label == 2:
            print("Tahmin edilen kategori: flamingo")
        elif predicted_label == 3:
            print("Tahmin edilen kategori: Motorbike")
        else:
            print("Hata")
    except Exception as knn_error:
        print(f"KNN tahmininde hata oluştu: {knn_error}")

except Exception as data_loading_error:
    print(f"!Veri yüklenirken hata oluştu: {data_loading_error}!")


Tahmin edilen kategori: Motorbike
